# 04. 영어/한국어 Word2Vec 훈련시키기

파이썬의 gensim 패키지에는 Word2Vec을 지원하고 있어, gensim 패키지를 이용하면 손쉽게 단어를 임베딩 벡터로 변환시킬 수 있습니다. 영어로 된 코퍼스를 다운받아 전처리를 수행하고, 전처리한 데이터를 바탕으로 Word2Vec 작업을 진행하겠습니다.

In [1]:
import nltk
nltk.download('punkt')
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sswwd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x172e67236a0>)

해당 파일은 xml 문법으로 작성되어 있어 자연어를 얻기 위해서는 전처리가 필요하다.

In [4]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
# 저자의 경우 윈도우 바탕화면에서 작업하여서 'C:\Users\USER\Desktop\ted_en-20160408.xml'이 해당 파일의 경로.  
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.

sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]

print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [5]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


## 3) Word2Vec 훈련시키기


In [9]:
!pip install gensim

In [11]:
from gensim.models import Word2Vec, KeyedVectors
model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

여기서 Word2Vec의 하이퍼파라미터값은 다음과 같습니다.
size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
window = 컨텍스트 윈도우 크기
min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
workers = 학습을 위한 프로세스 수
sg = 0은 CBOW, 1은 Skip-gram.

In [12]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8504380583763123), ('guy', 0.8252115845680237), ('lady', 0.7844919562339783), ('boy', 0.7633700966835022), ('girl', 0.7441701889038086), ('gentleman', 0.7377076148986816), ('soldier', 0.7263277173042297), ('kid', 0.6923007965087891), ('poet', 0.6832286715507507), ('person', 0.6540415287017822)]


# 한국어 word2vec 만들기

1. https://dumps.wikimedia.org/kowiki/latest/ 에 들어가서   
kowiki-latest-pages-articles.xml.bz2 파일 다운로드(약 40분걸림)  

2. https://github.com/attardi/wikiextractor 들어가서 code를 zip으로 다운로드  

3. 압축 풀고 cmd에서 파일 있는 경로로 가서 가상환경 설정해주고 'python setdup.py install'입력  

4. wikiextractor 폴더 안에 WikiExtractor.py 파일이 생기는데   
동일한 경로 안에 kowiki-latest-pages-articles.xml.bz2 넣어준다.  
* A:\chchdata\data\wikiextractor-master\wikiextractor\WikiExtractor.py  

* A:\chchdata\data\wikiextractor-master\wikiextractor\kowiki-latest-pages-articles.xml.bz2
5. cmd에 위의 경로로 들어가서 python -m wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2 입력

A:\chch\chch\pytorch로 시작하는 딥러닝 입문\8_단어의 표현 방법\4_영어,한국어 워드투벡터.ipynb

In [ ]:
python -m wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2